# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
#import csv created before
#file is in same filder as this ... opps
city_weather_data = pd.read_csv("city_weather.csv")

#print
city_weather_data.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Data
0,0,Dingle,11.00,122.67,83.88,81,48,9.22,PH,1605339589
1,1,Saint-Philippe,-21.36,55.77,75.20,78,90,16.11,RE,1605339486
2,2,Clyde River,70.47,-68.59,17.60,72,90,23.04,CA,1605339516
3,3,Qaanaaq,77.48,-69.36,1.58,81,91,4.94,GL,1605339332
4,4,Isangel,-19.55,169.27,80.60,83,75,5.82,VU,1605339553


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#6.3.9
#configurate gmaps
gmaps.configure(api_key=g_key)

In [10]:
#Use the Lat and Lng as locations and Humidity as the weight.
#Create a map using state centroid coordinates to set markers
marker_locations = city_weather_data[['Lat', 'Lng']]
Humidity = city_weather_data["Humidity"]


In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(marker_locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#create a new DataFrame fitting weather criteria with the following:
    #A max temperature lower than 80 degrees but higher than 70.
    #Wind speed less than 10 mph.
    #Zero cloudiness.

#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#do .loc instead
fitting_weather = city_weather_data.loc[(city_weather_data["Max Temp"] >=70) & (city_weather_data["Max Temp"] <= 80) & (city_weather_data["Wind Speed"] <= 10) & (city_weather_data["Cloudiness"] <= 0)]


#need to clean! to drop any nulls
fitting_weather = city_weather_data.dropna()
#SUCCESS!!

#print to see count - must be 500+ for a good data set
fitting_weather.count()

Unnamed: 0    544
City          544
Lat           544
Lng           544
Max Temp      544
Humidity      544
Cloudiness    544
Wind Speed    544
Country       544
Data          544
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
#store into variable named hotel_df
hotel_df = fitting_weather

#print
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Data
0,0,Dingle,11.00,122.67,83.88,81,48,9.22,PH,1605339589
1,1,Saint-Philippe,-21.36,55.77,75.20,78,90,16.11,RE,1605339486
2,2,Clyde River,70.47,-68.59,17.60,72,90,23.04,CA,1605339516
3,3,Qaanaaq,77.48,-69.36,1.58,81,91,4.94,GL,1605339332
4,4,Isangel,-19.55,169.27,80.60,83,75,5.82,VU,1605339553


In [16]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

#print
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Data,Hotel Name
0,0,Dingle,11.00,122.67,83.88,81,48,9.22,PH,1605339589,
1,1,Saint-Philippe,-21.36,55.77,75.20,78,90,16.11,RE,1605339486,
2,2,Clyde River,70.47,-68.59,17.60,72,90,23.04,CA,1605339516,
3,3,Qaanaaq,77.48,-69.36,1.58,81,91,4.94,GL,1605339332,
4,4,Isangel,-19.55,169.27,80.60,83,75,5.82,VU,1605339553,


In [17]:
 # Store latitude and longitude in locations
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# set up a parameters dictionary
#5000 radius
#6.3.1
#params = {
    #"keyword": "hotel",
    #"radius": 5000,
    #"type": "hotel",
    #"key": g_key

}

# base url
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
#response = requests.get(base_url, params=params)

In [37]:
# print the response url, avoid doing for public github repos in order to avoid exposing key
#print(response.url) 

In [20]:

params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]

    # add keyword to params dict
    params["location"] = f"{lat},{lng}" 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    #extract results
    #results = response['results']


    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")

hotel_df.head()

Missing field/result... skipping.


KeyboardInterrupt: 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig